# Pricing d'une option Bermudane sous Black–Scholes

Dans ce notebook, on met en place un **pricer Bermudan** (Bermudane) sous le modèle de Black–Scholes.

Objectifs&nbsp;:

1. Rappeler rapidement le cadre Black–Scholes.
2. Définir un contrat d'option Bermudane (put ou call) avec un nombre discret de dates d'exercice.
3. Implémenter un **arbre binomial** permettant :
   - de pricer une option européenne
   - de pricer une option américaine
   - de pricer une option bermudane (exercice uniquement sur un sous-ensemble de dates)
4. Comparer les trois prix : européenne, bermudane, américaine.

On reste volontairement en 1D et en Black–Scholes pour que tout soit transparent.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, suppress=True)

## 1. Paramètres du modèle et du contrat

On utilise une dynamique Black–Scholes sous la mesure risque-neutre :

\begin{equation}
dS_t = (r - q) S_t \, dt + \sigma S_t \, dW_t,
\end{equation}

avec :
- \( r \) : taux sans risque
- \( q \) : taux de dividende (ou repo)
- \( \sigma \) : volatilité
- \( S_0 \) : prix initial

Contrat :
- Call ou Put vanille standard
- Strike \( K \)
- Maturité \( T \)
- Dates d'exercice Bermudanes : sous-ensemble de la grille de l'arbre


In [ ]:
# Modèle
S0 = 100.0   # prix initial
K  = 100.0   # strike
r  = 0.02    # taux sans risque
q  = 0.00    # dividendes
sigma = 0.20 # volatilité
T  = 1.0     # maturité (en années)

# Arbre binomial
N_steps = 200  # nombre de pas de temps dans l'arbre

# Type d'option: 'call' ou 'put'
option_type = 'put'

## 2. Arbre binomial de Cox–Ross–Rubinstein (CRR)

On construit un arbre discret pour approximer la dynamique de Black–Scholes :

- Pas de temps : \( \Delta t = T / N \)
- Multiplicateurs :
  \[
  u = e^{\sigma \sqrt{\Delta t}}, \quad
  d = e^{-\sigma \sqrt{\Delta t}} = 1/u
  \]
- Probabilité risque-neutre :
  \[
  p = \frac{e^{(r - q)\Delta t} - d}{u - d}
  \]

À l'échéance, on initialise le payoff, puis on remonte l'arbre en appliquant l'actualisation risque-neutre
et la condition d'exercice selon le type d'option (européenne, américaine, bermudane).


In [ ]:
def build_crr_tree(S0, r, q, sigma, T, N):
    """Return u, d, p, dt, disc for a CRR binomial tree."""
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1.0 / u
    disc = np.exp(-r * dt)
    # risk-neutral prob
    p = (np.exp((r - q) * dt) - d) / (u - d)
    return u, d, p, dt, disc

In [ ]:
def payoff(S, K, option_type='call'):
    """Vanilla payoff for call or put."""
    if option_type == 'call':
        return np.maximum(S - K, 0.0)
    elif option_type == 'put':
        return np.maximum(K - S, 0.0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")

## 3. Pricing par arbre : Européenne, Américaine, Bermudane

On va écrire **une seule fonction** qui :
- construit les valeurs terminales de payoff,
- remonte l'arbre,
- applique la règle suivante à chaque nœud (temps \( t_i \)) :

- Européenne : continuation only
  \[
  V_i = e^{-r\Delta t} \big( p V_{i+1}^{up} + (1-p)V_{i+1}^{down} \big)
  \]

- Américaine : max continuation / exercice
  \[
  V_i = \max\Big( e^{-r\Delta t}\mathbb{E}[V_{i+1}], \ \text{payoff instantané} \Big)
  \]

- Bermudane : même chose, **mais uniquement aux dates d'exercice autorisées**.


In [ ]:
def price_binomial(S0, K, r, q, sigma, T, N, option_type='call',
                   style='european', bermudan_exercise_steps=None):
    """
    Price a vanilla option in a CRR binomial model.

    Parameters
    ----------
    style : 'european', 'american', or 'bermudan'
    bermudan_exercise_steps : iterable of time steps (integers) where early exercise is allowed
                              Only used if style == 'bermudan'. Steps go from 0..N (0 is t=0, N is maturity).

    Returns
    -------
    price : float
        Option price at t=0.
    """
    u, d, p, dt, disc = build_crr_tree(S0, r, q, sigma, T, N)

    # build stock prices at maturity: S_N(j) = S0 * u^j * d^(N-j)
    j = np.arange(N + 1)
    S_T = S0 * (u**j) * (d**(N - j))
    V = payoff(S_T, K, option_type)

    # backward induction
    # step index i: from N-1 down to 0
    for i in range(N - 1, -1, -1):
        # continuation value
        V = disc * (p * V[1:i+2] + (1-p) * V[0:i+1])

        if style == 'european':
            continue

        # stock prices at step i (length i+1)
        S_i = S0 * (u**np.arange(i + 1)) * (d**(i - np.arange(i + 1)))
        ex_val = payoff(S_i, K, option_type)

        if style == 'american':
            V = np.maximum(V, ex_val)
        elif style == 'bermudan':
            if bermudan_exercise_steps is not None and i in bermudan_exercise_steps:
                V = np.maximum(V, ex_val)
        else:
            raise ValueError("style must be 'european', 'american', or 'bermudan'")

    return V[0]

## 4. Exemple numérique : comparaison Euro / Bermuda / US

On fixe :
- un put (par exemple),
- un nombre total de pas `N_steps` dans l'arbre,
- un nombre de dates Bermudanes (par ex. exercice possible tous les 3 mois).

On compare ensuite :
- \(P_{Euro}\)
- \(P_{Bermuda}\)
- \(P_{American}\)

On doit avoir :
\[
P_{Euro} \le P_{Bermuda} \le P_{American}
\]


In [ ]:
# Construire des dates d'exercice Bermudanes: ex. tous les trimestres
N = N_steps
quarter_step = N // 4
bermudan_steps = [quarter_step, 2*quarter_step, 3*quarter_step, N]  # inclure la maturité par sécurité

print("Bermudan exercise steps:", bermudan_steps)

euro_price = price_binomial(S0, K, r, q, sigma, T, N, option_type, style='european')
amer_price = price_binomial(S0, K, r, q, sigma, T, N, option_type, style='american')
berm_price = price_binomial(S0, K, r, q, sigma, T, N, option_type, style='bermudan',
                            bermudan_exercise_steps=bermudan_steps)

print(f"European {option_type} price : {euro_price:.6f}")
print(f"Bermudan {option_type} price  : {berm_price:.6f}")
print(f"American {option_type} price  : {amer_price:.6f}")

## 5. Sensibilité au spot : comparaison des trois styles

On peut tracer le prix en fonction de \(S_0\) pour comparer :
- européenne
- bermudane
- américaine

et vérifier visuellement que la Bermudane se situe entre les deux.


In [ ]:
S0_grid = np.linspace(50, 150, 21)
euro_vals = []
amer_vals = []
berm_vals = []

for S_ in S0_grid:
    euro_vals.append(price_binomial(S_, K, r, q, sigma, T, N_steps, option_type, style='european'))
    amer_vals.append(price_binomial(S_, K, r, q, sigma, T, N_steps, option_type, style='american'))
    berm_vals.append(price_binomial(S_, K, r, q, sigma, T, N_steps, option_type, style='bermudan',
                                    bermudan_exercise_steps=bermudan_steps))

plt.plot(S0_grid, euro_vals, label='European')
plt.plot(S0_grid, berm_vals, label='Bermudan', linestyle='--')
plt.plot(S0_grid, amer_vals, label='American', linestyle=':')
plt.xlabel('S0')
plt.ylabel(f'Price ({option_type})')
plt.title('European vs Bermudan vs American')
plt.legend()
plt.grid(True)
plt.show()

## 6. Conclusion

- Une option Bermudane est **entre** l'Européenne et l'Américaine en termes de valeur.
- Il n'existe pas de **formule fermée** générale, mais un **schéma de backward induction**
  (arbre ou PDE) donne un prix de manière propre et contrôlée.
- Ce notebook montre un schéma simple (CRR) qui est déjà suffisant pour :
  - tester des idées,
  - comparer des structures,
  - générer des données pour du machine learning.

Améliorations possibles :
- passer sur un schéma trinomial plus stable,
- utiliser un schéma PDE implicite / Crank–Nicolson,
- traiter des Bermudanes multi-actifs via Monte Carlo + Longstaff–Schwartz.
